In [ ]:
支持非对称路径的pathsim（参见Hete-CF: Social-Based Collaborative Filtering Recommendation using Heterogeneous Relations）

In [1]:
from scipy.sparse import coo_matrix
import numpy as np
import pandas as pd
from tqdm import tqdm



# 数据处理示例
data_path="../data/ml-100k/u.data"
data_fields = ['user_id', 'item_id', 'rating', 'timestamp']
data_df = pd.read_table(data_path, names=data_fields)

# get user number
n_users = max(data_df['user_id'].values)
# get item number
n_items = max(data_df['item_id'].values)


data = np.ones((data_df.shape[0]))
data=data_df.rating.values
row = data_df.user_id-1
col = data_df.item_id-1

# 计数矩阵 pathCount
UI = coo_matrix((data, (row, col)), shape=(n_users, n_items))
UIU=UI.dot(UI.transpose())
IUI=UI.transpose().dot(UI)


In [2]:
K=10 # 取相似度最大的top10对象
def calSim(path, M, hinSim, hinSimI):
    hinSim[path]=np.empty((M.shape[0],K))
    hinSimI[path]=np.empty((M.shape[0],K),dtype=np.int)
    M=M.tocsc()
    col_sum={}
    for i in tqdm(range(M.shape[1])):
        col_sum[i]=M.getcol(i).toarray().sum()
    M=M.tocsr()
    row_sum={}
    for i in tqdm(range(M.shape[0])):
        row_sum[i]=M.getrow(i).toarray().sum()
    for i in tqdm(range(M.shape[0])):
        M_i=M.getrow(i).toarray()
        sim=[]
        for j in range(M_i.shape[1]):
            M_ij=M_i[0][j]
            M_i_=row_sum[i] 
            M_j_=col_sum[j]

            S_ij=2*M_ij/(M_i_+M_j_)
            sim.append(S_ij)
        sim=np.array(sim)
        ids=np.argsort(-sim)
        hinSimI[path][i]=ids[:K]
        hinSim[path][i]=sim[ids[:K]]


In [3]:
heteSim={}
heteSimI={}


calSim('UIU',IUI,heteSim, heteSimI)
calSim('IUI',IUI,heteSim, heteSimI)
calSim('UI',UI,heteSim, heteSimI)

100%|███████████████████████████████████████████████████████████████████████████████| 943/943 [00:01<00:00, 622.45it/s]


In [4]:
heteSim['UIU']

array([[0.00789809, 0.00610422, 0.00554021, ..., 0.00479463, 0.00469971,
        0.00466698],
       [0.0041556 , 0.00283757, 0.00274911, ..., 0.00256703, 0.00255534,
        0.00254744],
       [0.00460345, 0.00230217, 0.00221368, ..., 0.00192756, 0.00189306,
        0.001879  ],
       ...,
       [0.00718563, 0.00598802, 0.00399202, ..., 0.00242388, 0.00208578,
        0.00190331],
       [0.00278035, 0.00198577, 0.00188828, ..., 0.00059067, 0.00051078,
        0.0005034 ],
       [0.00281162, 0.0013132 , 0.0009564 , ..., 0.00040182, 0.00036791,
        0.00035051]])